# Lab10 -- Security in Computer  Networks (Part 2)

## Introduction

This lab aims to introduce you to security mechanisms used in computer networks. 
You will set up and analyze a VPN for secure network tunneling and a digital signature for sender authentication and message integrity.

For the final lab submission, provide a PDF report with answers to the questions labeled **REPORT** from labs 9 and 10. Questions marked with **Bonus** or **Extra** are optional.


## Lab Setup

[Figure 1](#figure_1) shows the setup for this lab.
You will set up a VPN to establish a secure tunnel between the two private networks.


<a id='figure_1'></a>

|<img src="figures/lab9.png" />|
|:--:| 
| *Figure 1: Labs 9 and 10 System Setup* |


- You can **open the GNS3 project from lab9** to view the topology, access the containers, and start packet captures. You can still use `docker attach` in your VM in order to access the containers, as long as the topology is started in GNS3.

In [ ]:
from test_lab10 import TestLab10
check_progress = TestLab10()

# Milestone 1 -- Virtual Private Network (VPN)


To establish a protected network over a public internet connection, we can set up a VPN to encrypt the traffic and allow access to services on private networks. For example, the "Openstack" VMs are only accessible via NTNU private network, and you had to set up a VPN connection to access them.

This lab will use WireGuard, free and open-source VPN software, which it is faster, simpler, and more efficient than IPsec and OpenVPN. It is already installed in all containers of this lab. You __must__ read this [page](https://www.wireguard.com/) (<https://www.wireguard.com/>) to get a conceptual overview of WireGuard.


* First, remove port forwarding from __"router0"__ to prevent external access to internal resources in "ttm4200_private_net":

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo nft flush ruleset</pre></div>


* In __"pclient2"__, verify that you can __not__ access the webserver:


<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">wget https://129.168.1.2 --header "Host: www.ttm4200.com" --no-check-certificate<br><br>wget https://10.20.30.3 --header "Host: www.ttm4200.com" --no-check-certificate
</pre></div>

In [ ]:
check_progress.test_1_0()

## Task 1.1 -- VPN Setup: WireGuard

In the following steps, you will establish a VPN connection from __"pclient2"__ (WireGuard client) to __"router0"__ (WireGuard server). 
Refer back to [Figure 1](#figure_1):


* Generate public and private keys for both the sever (__"router0"__) and the client (__"pclient2"__) (i.e., these commands need to be executed in both containers):

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">umask 077<br>wg genkey | tee privatekey | wg pubkey > publickey
</pre></div>

> More information about the command (<https://man7.org/linux/man-pages/man8/wg.8.html>):
>
> __- [umask 077](https://en.wikipedia.org/wiki/Umask)__: Set file permission for newly created files to only the user has "read," "write," and "execute" permissions (completely private).
>
> __- wg genkey__: Generates a random private key in base64.
>
> __- | tee privatekey__: Read the output of the generated private key and write it to a file named "privatekey" .
>
>__- | wg pubkey > publickey__: Calculates a public key from the corresponding private key and write it to a file named "publickey"


* In the server (__"router0"__), create a configuration file "/etc/wireguard/wg0.conf", and add the following lines:

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
[Interface]
PrivateKey = &lt;Private key of "router0"&gt;
<span style="color: #408080; font-style: italic">#We will use a VPN subnet of 10.20.30.128/25, use 10.20.30.130 for the server and 10.20.30.131 for the client.</span>
Address = 10.20.30.130/25
<span style="color: #408080; font-style: italic">#Standard port for WireGuard (optional)</span>
ListenPort = 51820
<span style="color: #408080; font-style: italic">#Accept every packet on the tunnel interface, and masquerade outgoing packets with a public IP</span>
PostUp = iptables -A FORWARD -i wg0 -j ACCEPT 
PostUP = iptables -t nat -A POSTROUTING -o ether1 -j MASQUERADE
<span style="color: #408080; font-style: italic">#Delete the NAT rule when removing the VPN</span>
PostDown = iptables -D FORWARD -i wg0 -j ACCEPT
PostDown = iptables -t nat -D POSTROUTING -o ether1 -j MASQUERADE

[Peer]
PublicKey = &lt;Public key of "pclient2"&gt;
AllowedIPs = 10.20.30.131
<span style="color: #408080; font-style: italic">#Send a keepalive packet every 25 seconds</span>
PersistentKeepalive = 25
</pre></div>

<div class="alert alert-block alert-info">
<b>Hint:</b> you can run <b>cat privatekey</b>, then copy the key and paste it into the config file. 
The same goes for the publickey </div>

* In the client (__"pclient2"__), create a configuration file "/etc/wireguard/wg0.conf", and add the following lines:

<div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">
[Interface]
PrivateKey = &lt;Private key of "pclient2"&gt;
Address = 10.20.30.131/25<br><br>
[Peer]
PublicKey = &lt;Public key of "router0"&gt;
<span style="color: #408080; font-style: italic">#Public ip address of the server "router0" and ListenPort</span>
Endpoint = 129.168.1.2:51820
<span style="color: #408080; font-style: italic">#Route all traffic with destination IP address in the subnet of (10.20.30.0/24) through this Wireguard tunnel</span>
AllowedIPs = 10.20.30.0/24
</pre></div>

* Bring up the WireGuard tunnel interface, in __both__ the server and the client:


   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo wg-quick up /etc/wireguard/wg0.conf</pre></div>


* Verify that there is a new interface "wg0" added:

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">ip address</pre></div>

* Ensure that the default route in __"webserver"__ is `10.20.30.100`:

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo ip route show</pre></div>

* Verify that your VPN is set up correctly by accessing the webserver from __"pclient2"__:

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">ping 10.20.30.3<br><br>wget https://10.20.30.3 --header "Host: www.ttm4200.com" --no-check-certificate</pre></div>

In [ ]:
check_progress.test_1_1()

## Task 1.2 -- Analyzing VPN

* Start packet capturing on the link connected to __"ether0"__ on __"router3"__ (as the outside party, or "Trudy"). Refer back to [Figure 1](#figure_1).


* Bring down WireGuard tunnel interface, then bring it up to capture the handshake (in __both__ the server and the client):

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo wg-quick down /etc/wireguard/wg0.conf<br><br>sudo wg-quick up /etc/wireguard/wg0.conf</pre></div>


* In __"pclient2"__, retrieve the content from the __"webserver"__, and then stop packet capturing.

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">wget https://10.20.30.3 --header "Host: www.ttm4200.com" --no-check-certificate</pre></div>

* Dump the capture to a file named "wireguard_https.pcap", and copy it to the "lab10" directory. 
* Apply the display filter `wg` in Wireshark to only show WireGuard packets. 

Then answer the following questions:

* Locate the "Handshake Initiation" packet, which corresponds to the first message from Initiator ("pclient2" in our case) to Responder ("router0" in our case). Then answer the following:
   * What is the packet number of the "Handshake Initiation" packet?
   * What is the used transport protocol? 
   * What is the destination port? 
   * __Self Check:__ Does the destination port corresponds to the "ListenPort" in the WireGuard server?
   * How many WireGuard fields are in the "Handshake Initiation" packet?
   * __Self Check:__ What are these fields? [HINT](https://www.wireguard.com/protocol/#first-message-initiator-to-responder)

* Locate the "Handshake Response" packet, which corresponds to the second message from Responder to Initiator. 
   * What is the packet number of the "Handshake Response" packet?
   * How many WireGuard fields are in the "Handshake Response" packet?
   * __Self Check:__ What are these fields? 


* Locate a "Transport Data" packet, corresponding to the encapsulated data.
   * What is the packet number of the "Transport Data" packet?
   * How many WireGuard fields are in the "Transport Data" packet?
   * __Self Check:__ What are these fields? 

* Locate a "Keepalive" message.   
   * What is the packet number of a "Keepalive" packet?
   * How many WireGuard fields are in the "Keepalive" packet?
   * __Self Check:__ What are these fields?

In [ ]:
################  Handshake Initiation ########################################
# What the packet number of the "Handshake Initiation" packet?
Handshake_Initiation_packet_number =  # your answer (as integer)

#  What is the used transport protocol? 
wireguard_transport_protocol= # your answer (as string)

# What is the destination port? 
Handshake_Initiation_destination_port= # your answer (as integer)

# How many WireGuard fields are in the "Handshake Initiation" packet?
Handshake_Initiation_num_fields =  # your answer (as integer)


################  Handshake Response ########################################
# What is the packet number of the "Handshake Response" packet?
Handshake_Response_packet_number =  # your answer (as integer)

# How many WireGuard fields are in the "Handshake Response" packet?
Handshake_Response_num_fields = # your answer (as integer)


################  Transport Data ########################################
# What is the packet number of a "Transport Data" packet?
Transport_Data_packet_number = # your answer (as integer)

# How many WireGuard fields are in the "Transport Data" packet?
Transport_Data_num_fields = # your answer (as integer)


################  Keepalive ########################################
# What is the packet number of a "Keepalive" packet?
Keepalive_packet_number = # your answer (as integer)

# How many WireGuard fields are in the "Keepalive" packet?
Keepalive_num_fields = # your answer (as integer)

check_progress.test_1_2(Handshake_Initiation_packet_number, 
                        wireguard_transport_protocol, 
                        Handshake_Initiation_destination_port,
                        Handshake_Initiation_num_fields,
                       Handshake_Response_packet_number,
                       Handshake_Response_num_fields,
                       Transport_Data_packet_number,
                       Transport_Data_num_fields,
                       Keepalive_packet_number,
                       Keepalive_num_fields)

  <span style="font-size: 25px;">Q3. </span> __REPORT:__ Which of the following security requirements does WireGuard provide/satisfy in this lab? Briefly explain why?
   * Confidentiality?
   * Integrity?
   * Authentication?
   * Forward Secrecy?
   * Traffic Type Obfuscation?

  <span style="font-size: 25px;">Q4. </span> __REPORT:__ 
   * What do you think would happen if you use the public IP address (i.e., `wget https://129.168.1.2 --header "Host: www.ttm4200.com" --no-check-certificate`) while using VPN? 
   * What would happen if you also perform port forwarding on "router0" (as you did in [section 1.1](#sec:1.1))? 
   * Finally, what would happen if you set the (“AllowedIPs = 0.0.0.0/0”) in the client (__"pclient2"__)?


__Optional:__ If you are interested, see how some of your colleagues used WireGuard in their master thesis [Link](https://ntnuopen.ntnu.no/ntnu-xmlui/bitstream/handle/11250/2834784/no.ntnu%3ainspera%3a80310926%3a20549418.pdf?sequence=1&isAllowed=y), [Link](https://ieeexplore.ieee.org/document/9289900).

__Extra:__ WireGuard supports optional Pre-shared Symmetric key Mode, which provides sufficient protection for post-quantum cryptography. According to the WireGuard protocol description: "If an additional layer of symmetric-key crypto is required (for, say, post-quantum resistance), WireGuard also supports an optional pre-shared key that is mixed into the public key cryptography. When the pre-shared key mode is not in use, the pre-shared key value is assumed to be an all-zero string of 32 bytes." (ref:<https://www.wireguard.com/protocol/>). Your __task__ is to reconfigure WireGuard with a pre-shared key (use the same setup as before, but include a pre-shared key). Submit a screenshot of your configuration, a screenshot of the output of `sudo wg show`. Briefly explain how using a Pre-shared key augments WireGuard security in case of quantum computing advances.

__Optional:__ [Interesting article](https://www.technologyreview.com/2021/11/03/1039171/hackers-quantum-computers-us-homeland-security-cryptography/).


# Milestone 2 -- Digital Signature


In this milestone, you will apply a digital signature to provide document authentication (verifying that a known sender created the document) and integrity (verifying that the document was unaltered in transit).
We will use [Pretty Good Privacy (PGP)](https://en.wikipedia.org/wiki/Pretty_Good_Privacy) protocol for creating and verifying signatures.

One common and free implementation of PGP is [GNU Privacy Guard (GPG)](https://en.wikipedia.org/wiki/GNU_Privacy_Guard), also known as [GnuPG](https://gnupg.org/).
We will use the command-line tool, which can be installed in your ntnu_server (no need for `docker` and GNS3 in this milestone):

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">sudo apt install gnupg</pre></div>

## Task 2.1 -- Public/Private Keys

* Create a PGP public/private key pair with the following command:

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">gpg --gen-key</pre></div>

Then enter a name (e.g., your team's name), email address (e.g., one of your emails), and a passphrase to protect your private key (__remember this passphrase!__).
By default, it will create an RSA key with 3072 bits.
You will use the private key to sign a document, while others can use your public key to verify your signature.

* Export your public key as an ASCII file so you can share it with others:


   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">gpg --export  youremail > teamName_public.key</pre></div>

* Import and trust others' public keys.
A public key belongs to this course at lab10 directory ("~/lab10/pgp/ttm4200_pubkey.asc"). 
You should import it, and trust it:

   <div style="background: #f8f8f8; overflow:auto;width:auto;border:solid gray;border-width:.1em .1em .1em .8em;padding:.2em .6em;"><pre style="margin: 0; line-height: 125%;color: black;">gpg --import ttm4200_pubkey.asc<br><br>gpg --edit-key ttm4200<br><br>trust<br>5<br>y<br>quit</pre></div>


## Task 2.2 -- Verifying Digital Signature

<span style="font-size: 25px;">Q5. </span> __REPORT:__ There is a pdf document at "&#126;/lab10/pgp/verify_me.pdf" that was digitally signed with the private key of ttm4200, where the signature file is at "&#126;/lab10/pgp/verify_me.pdf.sig". Verify the document's signature, then explain how this verification proves (or does not prove) the document's __integrity__ and __authenticity__.

<div class="alert alert-block alert-info">
<b>Hint:</b> Assume the public-key hasn't been tampered with. Check out this <a href="https://www.gnupg.org/gph/en/manual/x135.html" target="_blank">link</a>. </div>


## Task 2.3 -- Creating Digital Signature & Encryption

<span style="font-size: 25px;">Q6. </span> __REPORT:__ Explain how you can encrypt your report such that only the owner of the private key of ttm4200 can read the encryption (i.e., [how PGP encryption works?](https://en.wikipedia.org/wiki/Pretty_Good_Privacy#/media/File:PGP_diagram.svg)). Then (when you finish writing the delivery report), digitally sign your report (__detached signature__), and encrypt your report such that only the owner of the private key of ttm4200 can read the encryption. Submit the following files:

* Your report (unencrypted version).

* Your public key.

* The signature file.

* The encrypted report.